# Project 3: 'Image Analysis'
## First Year Project  
### ITU, Spring 2021

This notebook contains all the code developed to explore, wrangle and analyse the raw data sets for our project, 'Image Analysis'.

Contributors:  
- Andy Bao Nguyen (anbn)
- Florian Micliuc (flmi)
- Mattias Wohlert 
- Sofia Elena Terenziani (sote)

Created: 06-04-2021 

Last modified:

### Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm
from skimage import morphology
from scipy.spatial.distance import cdist
from scipy.stats.stats import mode
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [4]:
%run ../fyp2021p3_group00_functions.py

## ? Inspection of data

In [15]:
#We are using Micheals implementation from 'Lecture 1'
PATH = {}
PATH["data_raw"] = "../data/raw/"
PATH["data_interim"] = "../data/interim/"
PATH["data_processed"] = "../data/processed/"
PATH["data_external"] = "../data/external/"

PATH["images"] = "../data/raw/example_images"
PATH["segmentations"] = "../data/raw/example_segmentation"
PATH["images_features"] = "../features/"

FILENAME = {}
FILENAME["metadata"] = "example_ground_truth.csv"
FILENAME["images_features"] = "features.csv"

In [77]:
dataraw = {} 
dataraw["metadata"] = pd.read_csv(PATH["data_raw"]+FILENAME["metadata"])
dataraw["features"] = pd.read_csv(PATH["images_features"]+FILENAME["images_features"])

In [114]:
#merge datasets 
metadata = pd.DataFrame(dataraw["metadata"])
features = pd.DataFrame(dataraw["features"])
merged = metadata_pf.merge(features_pf, left_on = ["image_id"], right_on = ["id"])
merged = merged.drop(["id"], axis = 1)
print(merged)

         image_id  melanoma  seborrheic_keratosis        area  perimeter
0    ISIC_0001769       0.0                   0.0    216160.0     2013.0
1    ISIC_0001852       0.0                   0.0    130493.0     1372.0
2    ISIC_0001871       0.0                   0.0    205116.0     1720.0
3    ISIC_0003462       0.0                   0.0    161705.0     1344.0
4    ISIC_0003539       0.0                   0.0    317040.0     2063.0
..            ...       ...                   ...         ...        ...
145  ISIC_0015443       0.0                   0.0   9732988.0    13775.0
146  ISIC_0015445       0.0                   0.0  12049938.0    19517.0
147  ISIC_0015483       0.0                   0.0   8856478.0    13813.0
148  ISIC_0015496       0.0                   0.0   8951522.0    13742.0
149  ISIC_0015627       0.0                   0.0   3193093.0     8751.0

[150 rows x 5 columns]


In [115]:
#list of images splittet into skin desease
mask_melanoma = (merged["melanoma"] == 1.0) & (merged["seborrheic_keratosis"] == 0.0)
mask_keratosis = (merged["melanoma"] == 0.0) & (merged["seborrheic_keratosis"] == 1.0)
mask_none = (merged["melanoma"] == 0.0) & (merged["seborrheic_keratosis"] == 0.0)

merged_melanoma = merged[mask_melanoma]
merged_keratosis = merged[mask_keratosis]
merged_none = merged[mask_none]

melanoma_images_id = [i for i in merged_melanoma["image_id"]]
keratosis_images_id = [i for i in merged_keratosis["image_id"]]
none_images_id = [i for i in merged_none["image_id"]]


## Task 0: Explore the data